In [12]:

from io import BytesIO
import requests, re, pandas as pd, datetime as dt
from bs4 import BeautifulSoup

def get_data():
    t1=dt.datetime.now()
    url='https://www.dol.gov/agencies/eta/foreign-labor/performance'
    r=requests.get(url)
    soup=BeautifulSoup(r.content,'html.parser')
    res=soup.select('table.cust_table')
    final_links=[]
    keywords=['Appendix', 'Worksites', 'Icert']
    for i in res:
        if i.get('summary') == 'LCA Programs (H-1B, H-1B1, E-3)':
            i= i.find_all('a')
            for j in i:
                j=j.get('href')
                excel_sheet = re.search('.xlsx',j)
                key_search = re.search("|".join(re.escape(word) for word in keywords), j)
                if  excel_sheet is not None and key_search is None :
                    final_links.append(j)
    t2=dt.datetime.now()
    print(f'time to get the links from dol-website: {t2-t1}')
    return final_links

def dowload_data(final_links):
    for i,j in enumerate(final_links):
        # if i==4:
            t1=dt.datetime.now()
            sample = requests.get(f'https://www.dol.gov{j}')
            sample = BytesIO(sample.content)
            check = re.search(r'pdfs\/(.*)',j)
            checked = check.group().replace('pdfs/','')
            df=pd.read_excel(sample)
            df['linktofile'] = f'{checked}'
            temp_cols=df.columns.to_list()
            new_cols = temp_cols[-1:]+temp_cols[:-1]
            df=df[new_cols]
            # df=df.head()
            with open('dol-mega-file.json', mode='a', encoding='utf-8-sig') as json_file:
                df.to_json(json_file, orient='records', lines=True, date_format='iso',date_unit='ms')
            t2=dt.datetime.now()
            print(f'time it took for content#: {i}, is: {t2-t1}')

    return df

# print(dowload_data(get_data()))
final_links=get_data()
t1=dt.datetime.now()
sample = dowload_data(final_links)
t2=dt.datetime.now()

print(f'time to full operation: {t2-t1}')
# sample

time to get the links from dol-website: 0:00:00.658184
time it took for content#: 0, is: 0:02:47.688700
time it took for content#: 1, is: 0:03:32.203497
time it took for content#: 2, is: 0:05:11.019591
time it took for content#: 3, is: 0:01:21.256173
time it took for content#: 4, is: 0:01:25.643642
time it took for content#: 5, is: 0:01:48.109703
time it took for content#: 6, is: 0:03:23.103438
time it took for content#: 7, is: 0:01:24.393986
time it took for content#: 8, is: 0:01:03.646104
time it took for content#: 9, is: 0:03:57.914100
time it took for content#: 10, is: 0:07:25.124421
time it took for content#: 11, is: 0:01:42.526341
time it took for content#: 12, is: 0:01:29.374744
time it took for content#: 13, is: 0:02:04.579304
time it took for content#: 14, is: 0:02:29.964260
time it took for content#: 15, is: 0:01:37.501128
time it took for content#: 16, is: 0:12:34.668419
time it took for content#: 17, is: 0:04:48.213890
time it took for content#: 18, is: 0:05:29.749081
time 

In [15]:
df=pd.read_json(r'dol-mega-file.json', lines=True, encoding = 'utf-8-sig')
df

,linktofile,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,ORIGINAL_CERT_DATE,VISA_CLASS,JOB_TITLE,SOC_CODE,SOC_TITLE,...,NAME,ADDRESS1,ADDRESS2,CITY,STATE,POSTAL_CODE,CERTIFIED_BEGIN_DATE,CERTIFIED_END_DATE,JOB_CODE,WAGE_RATE_2
0,LCA_Disclosure_Data_FY2022_Q1.xlsx,I-200-21270-606997,Certified,2021-09-26T00:00:00.000,2021-10-01T00:00:00.000,None,H-1B,APPLICATIONS SUPPORT ANALYST/ADMINISTRATOR,15-1132.00,"Software Developers, Applications",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LCA_Disclosure_Data_FY2022_Q1.xlsx,I-200-21270-606867,Certified,2021-09-26T00:00:00.000,2021-10-01T00:00:00.000,None,H-1B,Designer,17-3013.00,Mechanical Drafters,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LCA_Disclosure_Data_FY2022_Q1.xlsx,I-200-21270-606846,Certified,2021-09-26T00:00:00.000,2021-10-01T00:00:00.000,None,H-1B,Data Analyst,15-2031.00,Operations Research Analysts,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LCA_Disclosure_Data_FY2022_Q1.xlsx,I-200-21270-606842,Certified,2021-09-26T00:00:00.000,2021-10-01T00:00:00.000,None,H-1B,Pharmaceutical Chemist,19-2031.00,Chemists,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,LCA_Disclosure_Data_FY2022_Q1.xlsx,I-200-21270-606941,Certified,2021-09-26T00:00:00.000,2021-10-01T00:00:00.000,None,H-1B,Senior Systems Analyst JC60,15-1121.00,Computer Systems Analysts,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,H-1B_Case_Data_FY2008.xlsx,NaN,NaN,NaN,NaN,NaN,NaN,PROGRAMMER ANALYST,NaN,NaN,...,"SPI RESEARCH AND DEVELOPMENT SERVICES, LLC",67 SOUTH BEDFORD STREET,205 W,BURLINGTON,MA,1803.0,NaN,NaN,30.0,NaN
141,H-1B_Case_Data_FY2008.xlsx,NaN,NaN,NaN,NaN,NaN,NaN,JAVA/J2EE DEVELOPER,NaN,NaN,...,"Nexus Software Solutions,Inc.","200 Lakeside Drive, Suite 228",None,Horsham,PA,19044.0,NaN,NaN,30.0,NaN
142,H-1B_Case_Data_FY2008.xlsx,NaN,NaN,NaN,NaN,NaN,NaN,COMPUTER PROGRAMMER,NaN,NaN,...,GREYCELL INC,3830 VILLAGE TER,SUITE 241,FREMONT,CA,94536.0,NaN,NaN,30.0,NaN
143,H-1B_Case_Data_FY2008.xlsx,NaN,NaN,NaN,NaN,NaN,NaN,PROGRAMMER ANALYST,NaN,NaN,...,DATA PIONEER INC,80 WALL STREET,SUITE # 912,NEW YORK,NY,10005.0,NaN,NaN,30.0,NaN


In [14]:
final_links

['/sites/dolgov/files/ETA/oflc/pdfs/LCA_Disclosure_Data_FY2023_Q4.xlsx',
 '/sites/dolgov/files/ETA/oflc/pdfs/LCA_Disclosure_Data_FY2023_Q3.xlsx',
 '/sites/dolgov/files/ETA/oflc/pdfs/LCA_Disclosure_Data_FY2023_Q2.xlsx',
 '/sites/dolgov/files/ETA/oflc/pdfs/LCA_Disclosure_Data_FY2023_Q1.xlsx',
 '/sites/dolgov/files/ETA/oflc/pdfs/LCA_Disclosure_Data_FY2022_Q1.xlsx',
 '/sites/dolgov/files/ETA/oflc/pdfs/LCA_Disclosure_Data_FY2022_Q2.xlsx',
 '/sites/dolgov/files/ETA/oflc/pdfs/LCA_Disclosure_Data_FY2022_Q3.xlsx',
 '/sites/dolgov/files/ETA/oflc/pdfs/LCA_Disclosure_Data_FY2022_Q4.xlsx',
 '/sites/dolgov/files/ETA/oflc/pdfs/LCA_Disclosure_Data_FY2021_Q1.xlsx',
 '/sites/dolgov/files/ETA/oflc/pdfs/LCA_Disclosure_Data_FY2021_Q2.xlsx',
 '/sites/dolgov/files/ETA/oflc/pdfs/LCA_Disclosure_Data_FY2021_Q3.xlsx',
 '/sites/dolgov/files/ETA/oflc/pdfs/LCA_Disclosure_Data_FY2021_Q4.xlsx',
 '/sites/dolgov/files/ETA/oflc/pdfs/LCA_Disclosure_Data_FY2020_Q1.xlsx',
 '/sites/dolgov/files/ETA/oflc/pdfs/LCA_Disclosure_

In [16]:
df.columns.to_list()

['linktofile',
 'CASE_NUMBER',
 'CASE_STATUS',
 'RECEIVED_DATE',
 'DECISION_DATE',
 'ORIGINAL_CERT_DATE',
 'VISA_CLASS',
 'JOB_TITLE',
 'SOC_CODE',
 'SOC_TITLE',
 'FULL_TIME_POSITION',
 'BEGIN_DATE',
 'END_DATE',
 'TOTAL_WORKER_POSITIONS',
 'NEW_EMPLOYMENT',
 'CONTINUED_EMPLOYMENT',
 'CHANGE_PREVIOUS_EMPLOYMENT',
 'NEW_CONCURRENT_EMPLOYMENT',
 'CHANGE_EMPLOYER',
 'AMENDED_PETITION',
 'EMPLOYER_NAME',
 'TRADE_NAME_DBA',
 'EMPLOYER_ADDRESS1',
 'EMPLOYER_ADDRESS2',
 'EMPLOYER_CITY',
 'EMPLOYER_STATE',
 'EMPLOYER_POSTAL_CODE',
 'EMPLOYER_COUNTRY',
 'EMPLOYER_PROVINCE',
 'EMPLOYER_PHONE',
 'EMPLOYER_PHONE_EXT',
 'NAICS_CODE',
 'EMPLOYER_POC_LAST_NAME',
 'EMPLOYER_POC_FIRST_NAME',
 'EMPLOYER_POC_MIDDLE_NAME',
 'EMPLOYER_POC_JOB_TITLE',
 'EMPLOYER_POC_ADDRESS1',
 'EMPLOYER_POC_ADDRESS2',
 'EMPLOYER_POC_CITY',
 'EMPLOYER_POC_STATE',
 'EMPLOYER_POC_POSTAL_CODE',
 'EMPLOYER_POC_COUNTRY',
 'EMPLOYER_POC_PROVINCE',
 'EMPLOYER_POC_PHONE',
 'EMPLOYER_POC_PHONE_EXT',
 'EMPLOYER_POC_EMAIL',
 'AGENT_RE